# Tut 4b - Hopper using Lagrange Mechanics
**Aim:** To model the monopod using a joint space formulation, which will hopefully be more stable and robust than the system space version.

## Generalized forces - a deeper dive
The reason I wanted to introduce you to the system space method, even though it results in a more finicky model, is that I personally find it much easier to conceptualize the way joints work when you're thinking about individual links in that classic Newtonian sort of way. Regardless of which coordinate set you decide to base your model around, working on interesting problems is probably still going to involve a lot of flipping between the world and joint frames, so in my opinion, you can only really get a strong understanding of the dynamics by being comfortable in both the Newtonian and the Lagrangian headspaces.

The thing I find personally tricky about representing joints (and the action of forces more broadly) using Lagrange mechanics is that the concept of direction becomes a little hazy, and can be confusing when you try to project it from joint coordinates back into world coordinates. Take our familiar old pendulum, for example:

<img src = "pendulum_force2.png" width = "300">

We actuate the second joint by introducing a torque acting in the direction of $\theta_2$. In joint space, this makes perfect sense: positive $\tau$ acts to increase the angle $\theta_2$. But because $\theta_2$ is a relative angle, the direction of positive $\tau$ can only be explained in the absolute world frame in terms of its effects on individual links: it rotates link 1 clockwise and link 2 anticlockwise.

The prismatic joint is even trickier, because it introduces a bit of ambiguity about which coordinates should be affected at all. Consider the following system, where we've simplified the problem by reducing it to only the $x$ dimension in the world frame:

<img src = "prismatic1D.png" width = "400">

We decide our generalized coordinates should be the position of the first link's COM, $x_1$, and the length of the joint, $r$. The force $F$ acts to increase $r$, but wouldn't it also push the first link backwards and therefore influence $x_1$?

I've already given you a quick and dirty explanation of generalized forces that's probably sufficient for making things work, but it wasn't strictly correct in some ways. To truly *get* these two examples, we need to dig deeper into the details.

### Clarifying the terminology
**WTF is a virtual displacement, even?**
The virtual displacement isn't actually used to calculate the generalized force, but it might still be beneficial to understand what it is, and how it relates to the quantities we do actually use.

The Wikipedia definition (paraphrased) is that a virtual displacement is "an infinitesimally small change of coordinates occurring while time is held constant." The "virtual" part refers to the fact that no actual displacement can happen without time passing. It's only a concept that makes sense and becomes useful in the context of constrained motion - rather than representing any actual motion that happens, its purpose is to indicate the directions in which motion *can* happen. 

To poach from Wikipedia further, imagine a bead sliding along a circular hoop. A good generalized coordinate for its position would be $\mathbf{q} = \theta$. The bead is able to move in the direction $\theta$, represented by the virtual displacement $\delta \theta$. But what about in the world frame? 

<img src = "hoop.png" width = "300">

The world position of the bead is given by the coordinates $[x,y]$. The link between the two coordinate frames is the vector $\mathbf{r} = [R cos(\theta), R sin(\theta)]$, which represents the world position in terms of the generalized coordinates (more on this guy in a second). The virtual displacement in world terms is given by $$\mathbf{\delta r} = \sum_{j} \frac{\partial \mathbf{r}}{\partial q_j} \delta q_j$$

The loop only has one generalized coordinate, so the only $q_j$ we have to think about is $\theta$. Thus, $$\mathbf{\delta r} = \frac{\partial \mathbf{r}}{\partial \theta} \delta \theta = [-R sin(\theta)\delta \theta, R cos(\theta) \delta \theta]$$ which clearly indicates that the bead is constrained to only move in a direction tangent to the circle.

**More on the position vector.** Previously, I said $\mathbf{r}$ is the position in the world frame where the force acts expressed in terms of the generalized coordinates. That was the 'technically not correct but still works-ish' part of my previous explanation. 

It's actually the world-frame position of *the whole system* as a function of the generalized coordinates. It's not even really a vector when you think of it that way, or rather, it's not a single vector: you need one for each rigid body in your system. 

So for each link $i$, you have an $\mathbf{r_i} = [x_i(\mathbf{q}), y_i(\mathbf{q}), \theta_i(\mathbf{q})]$.

(Or, if you're not a flatlander, $\mathbf{r_i} = [x_i(\mathbf{q}), y_i(\mathbf{q}), z_i(\mathbf{q}), \theta_i(\mathbf{q}), \phi_i(\mathbf{q}), \psi_i(\mathbf{q})]$)

**Calculating the generalized force - a better explanation:**
Say you want to calculate the component of some action $F$ acting in the direction of a generalized coordinate $q_j$. 

(I'm going to refer to this component as $Q_j$, but beware that this notation actually refers to the *total* force in the direction of that coordinate. So if you had a bunch of actions going on, of which $F_k$ is just one, $Q_j = \sum_{k}Q_{j,k}F_k$. But I don't feel like getting bogged down in subscript Hell, so let's just go with $Q_j$.)

The formula is $$Q_j = \sum_{i}\mathbf{F_i} \cdot \frac{\partial \mathbf{r_i}}{\partial q_j}$$

where $\mathbf{F_i}$ is the world-frame vector expressing how $F$ affects the $i$th link. So, $Q_j$ is a combination of the effects of the force on all of the individual bodies in the system. 

### Example 1: the pendulum, revisited

For link 1, $\mathbf{r_1} = \begin{pmatrix} \frac{1}{2}l_1 sin(\theta_1) 
\\ -\frac{1}{2}l_1 cos(\theta_1) 
\\ \theta_1 \end{pmatrix}$ and for link 2, $\mathbf{r_2} = \begin{pmatrix} l_1 sin(\theta_1) + \frac{1}{2}l_2 sin (\theta_1 + \theta_2) 
\\-l_1 cos(\theta_1) -\frac{1}{2}l_2 cos(\theta_2)
\\ \theta_1 + \theta_2 \end{pmatrix}$.

I'll leave calculating the Jacobians for each link up to you. (Remember, $\frac{\partial \mathbf{r_i}}{\partial q_j}$ is the $j$th column of the Jacobian.)

**The horizontal force:**

$\mathbf{F_1} = \begin{pmatrix} 0\\0\\0 \end{pmatrix}$ because the force doesn't act on link 1.

$\mathbf{F_2} = \begin{pmatrix} F\\0\\F \frac{1}{2}l_2 cos(\theta_1+\theta_2) \end{pmatrix}$

$Q_{\theta_1} = \mathbf{F_1} \cdot \frac{\partial \mathbf{r_1}}{\partial \theta_1} + \mathbf{F_2} \cdot \frac{\partial \mathbf{r_2}}{\partial \theta_1} = F l_1 cos(\theta_1) + F l_2 cos(\theta_1 + \theta_2)$

And $Q_{\theta_2} = \mathbf{F_1} \cdot \frac{\partial \mathbf{r_1}}{\partial \theta_2} + \mathbf{F_2} \cdot \frac{\partial \mathbf{r_2}}{\partial \theta_2} = F l_2 cos(\theta_1 + \theta_2)$

**The rotary joint:**

$\mathbf{\tau_1} = \begin{pmatrix} 0\\0\\-\tau \end{pmatrix}$ 

$\mathbf{\tau_2} = \begin{pmatrix} 0\\0\\ \tau \end{pmatrix}$

Do thing, get $Q_{\theta_1} = -\tau + \tau = 0$ and $Q_{\theta_2} = \tau$.

### Example 2: prismatic joint

Using $x$ for the COM of the left link, $r_1 = x$ and $r_2 = x + r$. (Yes, I'm still using $r$ for the length of the joint. Any confusion is everybody else's fault for deciding $\mathbf{r_i}$ is a sensible symbol for world-frame position.) 

$F_1 = -F$ and $F_2 = F$.

Do the thing, get $Q_{x} = -F + F = 0$ and $Q_{r} = F$.

Now, one vast digression later, we're ready to do this for a 2D system in the code.

In [ ]:
%reset
# DERIVE THE EOMs SYMBOLICALLY ----------------------------------------------------------------------------------------------

# import libraries
import sympy as sym
import numpy as np

sym.init_printing()
from IPython.display import display #for pretty printing

# create symbolic variables

# system parameters
g = sym.symbols('g')
mb,ml1,ml2 = sym.symbols(['m_{body}','m_{leg1}','m_{leg2}']) # mass
lb,ll1,ll2 = sym.symbols(['l_{body}','l_{leg1}','l_{leg2}']) # length
Inb,Inl1,Inl2 = sym.symbols(['I_{body}','I_{leg1}','I_{leg2}']) # moment of intertia

# generalized coordinates
x,y,thb,thl,r = sym.symbols(['x','y','\\theta_{body}','\\theta_{leg}','r']) 
dx,dy,dthb,dthl,dr = sym.symbols(['\dot{x}','\dot{y}','\dot{\\theta}_{body}','\dot{\\theta}_{leg}','\dot{r}']) 
ddx,ddy,ddthb,ddthl,ddr = sym.symbols(['\ddot{x}','\ddot{y}','\ddot{\\theta}_{body}','\ddot{\\theta}_{leg}','\ddot{r}']) 

q = sym.Matrix([[x],[y],[thb],[thl],[r]])
dq = sym.Matrix([[dx],[dy],[dthb],[dthl],[dr]])
ddq = sym.Matrix([[ddx],[ddy],[ddthb],[ddthl],[ddr]])

# forces
# total joint action = actuator + rebound, but that will be dealt with elsewhere
F,tau,GRFx,GRFy = sym.symbols(['F','\\tau','G_x','G_y']) 

# STEP 1: position vectors ri = [x,y,theta] (world frame)
rb = sym.Matrix([[x],
                [y],
                [thb]])

rl1 = sym.Matrix([[x + 0.5*ll1*sym.sin(thb + thl)],
                [y - 0.5*ll1*sym.cos(thb + thl)],
                [thb + thl]])

rl2 = sym.Matrix([[x + (0.5*ll1+r)*sym.sin(thb + thl)],
                [y - (0.5*ll1+r)*sym.cos(thb + thl)],
                [thb + thl]])

# the Jacobians
Jb = rb.jacobian(q)
Jl1 = rl1.jacobian(q)
Jl2 = rl2.jacobian(q)

# STEP 2: generate expressions for the system space velocities from the jacobians
vb = Jb*dq
vl1 = Jl1*dq
vl2 = Jl2*dq

# STEP 3: generate expressions for the kinetic and potential energy
# mass vectors
Mb = sym.Matrix([[mb,mb,Inb]])
Ml1 = sym.Matrix([[ml1,ml1,Inl1]])
Ml2 = sym.Matrix([[ml2,ml2,Inl2]])

T = 0.5*Mb*sym.matrix_multiply_elementwise(vb,vb) + 0.5*Ml1*sym.matrix_multiply_elementwise(vl1,vl1) + 0.5*Ml2*sym.matrix_multiply_elementwise(vl2,vl2)
T = T[0]
V = mb*g*rb[1] + ml1*g*rl1[1] + ml2*g*rl2[1]


# STEP 4: calculate each term of the Lagrange equation
# term 1
Lg1 = sym.zeros(len(q),1)
for i in range(len(q)):
    dT_ddq = sym.Matrix([sym.diff(T,dq[i])]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = sym.Matrix([T]).jacobian(q).transpose() # partial of T in q

# term 4
Lg4 = sym.Matrix([V]).jacobian(q).transpose() # partial of U in q

# STEP 5: generalized forces

# force vectors for each link
tau_b = sym.Matrix([[0],[0],[-tau]])
tau_l1 = sym.Matrix([[0],[0],[tau]])

F_l1 = sym.Matrix([[-F*sym.sin(thb+thl)],[F*sym.cos(thb+thl)],[0]])
F_l2 = sym.Matrix([[F*sym.sin(thb+thl)],[-F*sym.cos(thb+thl)],[0]])

GRF_l2 = sym.Matrix([[GRFx],[GRFy],[-GRFx*sym.cos(thb+thl)-GRFy*sym.sin(thb+thl)]])

Q = sym.zeros(len(q),1)
for j in range(len(q)):
    Q[j] = tau_b.transpose()*Jb[:,j]+(tau_l1+F_l1).transpose()*Jl1[:,j]+(F_l2+GRF_l2).transpose()*Jl2[:,j]

# AND combine!
EOM = Lg1 - Lg3 + Lg4 - Q

EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    EOMs[j] = EOM[j].simplify()
    
# the simplification step is a little time-consuming so try to avoid re-running this cell if possible.

In [ ]:
# Lambdify
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

func_map = {'sin':sin, 'cos':cos} 

sym_list = [g,mb,ml1,ml2,lb,ll1,ll2,Inb,Inl1,Inl2,
            x,y,thb,thl,r,
            dx,dy,dthb,dthl,dr,
            ddx,ddy,ddthb,ddthl,ddr,
            F,tau,GRFx,GRFy]
            
lambEOM_x = sym.lambdify(sym_list,EOMs[0],modules = [func_map])
lambEOM_y = sym.lambdify(sym_list,EOMs[1],modules = [func_map])
lambEOM_thb = sym.lambdify(sym_list,EOMs[2],modules = [func_map])
lambEOM_thl = sym.lambdify(sym_list,EOMs[3],modules = [func_map])
lambEOM_r = sym.lambdify(sym_list,EOMs[4],modules = [func_map])

In [ ]:
# rerun from here if you don't want to calculate the EOM's again
if 'm' in globals():
    del m # deletes the model
    
m = ConcreteModel()

# SETS-----------------------------------------------------------------------------------------------------------------------

N = 80
m.N = RangeSet(N) 

links = [('body',1),('leg',1),('leg',2)]
m.L = Set(dimen=2, initialize = links)

DOFs = ['x','y','theta_b','theta_l','r'] # generalized coordinates
m.DOF = Set(initialize = DOFs) 

# PARAMETERS-----------------------------------------------------------------------------------------------------------------

m.g = Param(initialize = 9.81)

def get_m(n, lb, ln):
    if lb == 'body':
        return 5.0
    else: return 2.5
m.m = Param(m.L, initialize = get_m) # mass of links

def get_len(n, lb, ln):
    if lb == 'body':
        return 1.0
    else: return 0.5
m.len = Param(m.L, initialize = get_len) # length of links

def calculate_In(m, lb, ln): 
    l = (lb,ln)
    # yes, that does mean you have to rebuild the tuple inside the function. Yes, it is dumb.
    return m.m[l]*m.len[l]**2/12 
m.In = Param(m.L, initialize = calculate_In) # moment of inertia

mbody = sum(m.m[l] for l in links)
BW = mbody*m.g.value

# VARIABLES -----------------------------------------------------------------------------------------------------------------

# system coordinates
m.q = Var(m.N, m.DOF) # position
m.dq = Var(m.N, m.DOF) # velocity
m.ddq = Var(m.N, m.DOF) # acceleration

# bound variables
for n in range(1,N+1):
    for l in links:
        m.q[n,'y'].setlb(0.0)

In [ ]:
# TIME AND INTEGRATION

# variable timestep
hm = 0.02 # master timestep
m.h = Var(m.N, bounds = (0.8,1.2))

# Integration constraints 
def BwEuler_p(m,n,dof): # for positions
    if n > 1:
        return m.q[n,dof] == m.q[n-1,dof] + hm*m.h[n]*m.dq[n,dof]
    else:
        return Constraint.Skip #use this to leave out members of a set that the constraint doesn't apply to
m.integrate_p = Constraint(m.N, m.DOF, rule = BwEuler_p)

def BwEuler_v(m,n,dof): # for velocities
    if n > 1:
        return m.dq[n,dof] == m.dq[n-1,dof] + hm*m.h[n]*m.ddq[n-1,dof]
    else:
        return Constraint.Skip 
m.integrate_v = Constraint(m.N, m.DOF, rule = BwEuler_v)

### Protip: lambdify your aux variable definitions
As you add links to the leg, the expressions for the positions, and especially the velocities of the contacts get very unweildly very quickly. Luckily, the LORD hath given unto us sympy, that we need fear no kerfuffle. Blessed be the name of the LORD, *amen*. I use it to get the horizontal foot velocity.

In [ ]:
# GROUND INTERACTIONS -------------------------------------------------------------------------------------------------------

# paramters
m.mu = Param(initialize = 1) # friction coefficient

# sign set for positive and negative components
signs = ['ps','ng'] 
m.sgn = Set(initialize = signs)

WDOFs = ['X','Y',"THETA"] # absolute coordinates (see what I mean about switching between frames the whole time...?)
m.WDOF = Set(initialize = WDOFs) 

# variables
m.footp = Var(m.N, m.WDOF, bounds = (0.0,None)) # foot position
m.footv = Var(m.N, m.WDOF, m.sgn, bounds = (0.0,None)) # foot velocity

m.friction_cone = Var(m.N, bounds = (0.0,None))

m.GRF = Var(m.N, m.WDOF, m.sgn, bounds = (0.0,None)) # ground reaction forces

ground_constraints = ['contact','friction','slip_ps','slip_ng'] 
m.ground_constraints = Set(initialize = ground_constraints) # set for indexing ground-related penalties
m.ground_penalty = Var(m.N, m.ground_constraints, bounds = (0.0,None))

# constraints: aux variables
def def_footp(m,n,dof):
    if dof == 'Y':
        L = 0.5*m.len[('leg',1)] + m.q[n,'r'] + 0.5*m.len[('leg',2)] # total leg length
        thA = m.q[n,'theta_b'] + m.q[n,'theta_l'] # absolute leg angle
        return m.footp[n,dof] == m.q[n,'y'] - L*cos(thA)
    else:
        return Constraint.Skip
m.def_footp = Constraint(m.N, m.WDOF, rule = def_footp)

# lambdify the foot velocity
footx = sym.Matrix([x + (0.5*ll1+r+0.5*ll2)*sym.sin(thb + thl)])
footdx = footx.jacobian(q)*dq
footdx = footdx[0].simplify()
lamb_footdx = sym.lambdify(sym_list,footdx,modules = [func_map])

def def_footv(m,n,dof):
    if dof == 'X':
        var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_l'],m.q[n,'r'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_l'],m.dq[n,'r'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_l'],m.ddq[n,'r'],
            0,0,0,0]
        return m.footv[n,dof,'ps']-m.footv[n,dof,'ng'] == lamb_footdx(*var_list)
    else:
        return Constraint.Skip
m.def_footv = Constraint(m.N, m.WDOF, rule = def_footv)

def def_friction_cone(m,n):
    return m.friction_cone[n] == m.mu*m.GRF[n,'Y','ps'] - (m.GRF[n,'X','ps'] + m.GRF[n,'X','ng'])
m.def_friction_cone = Constraint(m.N, rule = def_friction_cone)

# constraints: complementarity

# contact
def ground_contact(m,n):
    if n < N:
        return m.ground_penalty[n,'contact'] == m.footp[n+1,'Y']*m.GRF[n,'Y','ps'] 
    else:
        return Constraint.Skip
m.ground_contact = Constraint(m.N, rule = ground_contact)

# friction
def ground_friction(m,n):
    return m.ground_penalty[n,'friction'] == (m.footv[n,'X','ps']+m.footv[n,'X','ng'])*m.friction_cone[n]
m.ground_friction = Constraint(m.N, rule = ground_friction)

# slipping
def ground_slip_ps(m,n):
    return m.ground_penalty[n,'slip_ps'] == m.footv[n,'X','ps']*m.GRF[n,'X','ps']
m.ground_slip_ps = Constraint(m.N, rule = ground_slip_ps)

def ground_slip_ng(m,n):
    return m.ground_penalty[n,'slip_ng'] == m.footv[n,'X','ng']*m.GRF[n,'X','ng']
m.ground_slip_ng = Constraint(m.N, rule = ground_slip_ng)

# bound contact forces at last node
for dof in WDOFs:
    for sgn in signs:
        m.GRF[N,dof,sgn].value = 0
        m.GRF[N,dof,sgn].fixed = True

In [ ]:
# HARD JOINT STOPS ----------------------------------------------------------------------------------------------------------

# sets
joints = ['hip','knee']
m.J = Set(initialize = joints)

joint_constraints = ['up','lo'] # set of joint penalties
m.joint_constraints = Set(initialize = joint_constraints)

# parameters
hip_bound = [-np.pi/2,np.pi/2]
m.hip_bound = Param(m.joint_constraints, initialize = {'up':hip_bound[1],'lo':hip_bound[0]}) 

knee_bound = [0.0,0.5]
m.knee_bound = Param(m.joint_constraints, initialize = {'up':knee_bound[1],'lo':knee_bound[0]})

# we can bound the joint coordinates directly
for n in range(1,N+1):
    m.q[n,'theta_l'].setlb(hip_bound[0])
    m.q[n,'theta_l'].setub(hip_bound[1])
    m.q[n,'r'].setlb(knee_bound[0])
    m.q[n,'r'].setub(knee_bound[1])

# variables
m.tau_a = Var(m.N, bounds = (-2,2)) # actuator torque at hip
m.tau_r = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound torque

m.F_a = Var(m.N, bounds = (-3,3)) # actuator force at knee
m.F_r = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound force (acts parallel to the leg)

m.joint_penalty = Var(m.N, m.J, m.joint_constraints, bounds = (0.0,None))

# complementarity
def hip_limits(m,n,jc):
    if n < N:
        if jc == 'up':
            # NEXT angle
            return m.joint_penalty[n,'hip',jc] == (m.hip_bound['up'] - m.q[n+1,'theta_l'])*m.tau_r[n,'ng']
        else:
            return m.joint_penalty[n,'hip',jc] == (m.q[n+1,'theta_l'] - m.hip_bound['lo'])*m.tau_r[n,'ps']
    else:
        return Constraint.Skip
m.hip_limits = Constraint(m.N, m.joint_constraints, rule = hip_limits)

def knee_limits(m,n,jc):
    if n < N:
        if jc == 'up':
            # NEXT distance
            return m.joint_penalty[n,'knee',jc] == (m.knee_bound['up'] - m.q[n+1,'r'])*m.F_r[n,'ng']
        else:
            return m.joint_penalty[n,'knee',jc] == (m.q[n+1,'r'] - m.knee_bound['lo'])*m.F_r[n,'ps']
    else:
        return Constraint.Skip
m.knee_limits = Constraint(m.N, m.joint_constraints, rule = knee_limits)

# bound contact forces at last node
for sgn in signs:
    m.F_r[N,sgn].value = 0
    m.F_r[N,sgn].fixed = True

for sgn in signs:
    m.tau_r[N,sgn].value = 0
    m.tau_r[N,sgn].fixed = True

In [ ]:
# COST FUNCTION -------------------------------------------------------------------------------------------------------------

# minimum time

def CostFun(m):
    T = sum(m.h[n] for n in range(1,N+1))
    penalty_sum = 0
    for n in range(1,N+1):
        for gc in ground_constraints:
            penalty_sum += m.ground_penalty[n,gc]
        for jc in joint_constraints:
            for j in joints:
                penalty_sum += m.joint_penalty[n,j,jc]
    return T+1000*penalty_sum
m.Cost = Objective(rule = CostFun)

In [ ]:
# EQUATIONS OF MOTION -------------------------------------------------------------------------------------------------------
S = BW

def EOM_x(m,n):
    F_in = S*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_in = S*(m.tau_a[n] + m.tau_r[n,'ps'] - m.tau_r[n,'ng'])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_l'],m.q[n,'r'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_l'],m.dq[n,'r'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_l'],m.ddq[n,'r'],
            F_in,tau_in,Gx_in,Gy_in]
    return lambEOM_x(*var_list) == 0
m.EOM_x = Constraint(m.N, rule = EOM_x)

def EOM_y(m,n):
    F_in = S*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_in = S*(m.tau_a[n] + m.tau_r[n,'ps'] - m.tau_r[n,'ng'])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_l'],m.q[n,'r'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_l'],m.dq[n,'r'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_l'],m.ddq[n,'r'],
            F_in,tau_in,Gx_in,Gy_in]
    return lambEOM_y(*var_list) == 0
m.EOM_y = Constraint(m.N, rule = EOM_y)

def EOM_thb(m,n):
    F_in = S*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_in = S*(m.tau_a[n] + m.tau_r[n,'ps'] - m.tau_r[n,'ng'])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_l'],m.q[n,'r'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_l'],m.dq[n,'r'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_l'],m.ddq[n,'r'],
            F_in,tau_in,Gx_in,Gy_in]
    return lambEOM_thb(*var_list) == 0
m.EOM_thb = Constraint(m.N, rule = EOM_thb)

def EOM_thl(m,n):
    F_in = S*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_in = S*(m.tau_a[n] + m.tau_r[n,'ps'] - m.tau_r[n,'ng'])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_l'],m.q[n,'r'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_l'],m.dq[n,'r'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_l'],m.ddq[n,'r'],
            F_in,tau_in,Gx_in,Gy_in]
    return lambEOM_thl(*var_list) == 0
m.EOM_thl = Constraint(m.N, rule = EOM_thl)

def EOM_r(m,n):
    F_in = S*(m.F_a[n] + m.F_r[n,'ps'] - m.F_r[n,'ng'])
    tau_in = S*(m.tau_a[n] + m.tau_r[n,'ps'] - m.tau_r[n,'ng'])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_l'],m.q[n,'r'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_l'],m.dq[n,'r'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_l'],m.ddq[n,'r'],
            F_in,tau_in,Gx_in,Gy_in]
    return lambEOM_r(*var_list) == 0
m.EOM_r = Constraint(m.N, rule = EOM_r)

In [ ]:
# # HIGH DROP -----------------------------------------------------------------------------------------------------------------

# # initial condition
# m.q[1,'x'].value = 0.0
# m.q[1,'y'].value = 3.0
# m.q[1,'theta_b'].value = 0.0
# m.q[1,'theta_l'].value = 0.0
# m.q[1,'r'].value = 0.25

# for dof in DOFs:
#     m.q[1,dof].fixed = True
#     m.dq[1,dof].value = 0.0
#     m.dq[1,dof].fixed = True
    
# for n in range(1,N+1):
#     m.F_a[n].value = 0.0
#     m.F_a[n].fixed = True
#     m.tau_a[n].value = 0.0
#     m.tau_a[n].fixed = True
    
# #m.pprint()

In [ ]:
# # LOW DROP ------------------------------------------------------------------------------------------------------------------

# # initial condition
# m.q[1,'x'].value = 0.0
# m.q[1,'theta_b'].value = 0.0
# m.q[1,'theta_l'].value = 0.0
# m.q[1,'r'].value = 0.25

# m.footp[1,'Y'].value = 0.1
# m.footp[1,'Y'].fixed = True

# for dof in DOFs:
#     if dof != 'y':
#         m.q[1,dof].fixed = True
#     m.dq[1,dof].value = 0.0
#     m.dq[1,dof].fixed = True
    
# for n in range(1,N+1):
#     m.F_a[n].value = 0.0
#     m.F_a[n].fixed = True
#     m.tau_a[n].value = 0.0
#     m.tau_a[n].fixed = True
    
# #m.pprint()

In [ ]:
# # STANDING ------------------------------------------------------------------------------------------------------------------

# # initial condition
# m.q[1,'x'].value = 0.0
# m.q[1,'theta_b'].value = 0.0
# m.q[1,'theta_l'].value = 0.0
# m.q[1,'r'].value = 0.25

# m.footp[1,'Y'].value = 0.0
# m.footp[1,'Y'].fixed = True

# for dof in DOFs:
#     if dof != 'y':
#         m.q[1,dof].fixed = True
#     m.dq[1,dof].value = 0.0
#     m.dq[1,dof].fixed = True
    
# for n in range(1,N+1):
#     m.F_a[n].value = 0.0
#     m.F_a[n].fixed = True
#     m.tau_a[n].value = 0.0
#     m.tau_a[n].fixed = True
    
# #m.pprint()

In [ ]:
# # HOP -----------------------------------------------------------------------------------------------------------------------

# # initial condition
# m.q[1,'x'].value = 0.0
# m.q[1,'theta_b'].value = 0.0
# m.q[1,'theta_l'].value = 0.0
# m.q[1,'r'].value = 0.25

# m.footp[1,'Y'].value = 0.0
# m.footp[1,'Y'].fixed = True

# for dof in DOFs:
#     if dof != 'y':
#         m.q[1,dof].fixed = True
#     m.dq[1,dof].value = 0.0
#     m.dq[1,dof].fixed = True
    
# # final condition
# m.q[N,'y'].setlb(1.2)
    
# #m.pprint()

### Random ground reaction forces
I often find it beneficial to randomize the ground reaction forces, assuming the model is stable enough to handle it. Sometimes if I leave the GRF's uninitialized, the model appears to be 'avoiding' the ground - obviously it doesn't do this consciously, I suspect it just gets stuck in a region of the solution space where the GRFs are mostly zero. It's something worth trying if your model appears to think a flying leap is the best motion for all situations.

In [ ]:
# INITIALIZE ----------------------------------------------------------------------------------------------------------------
guide = np.linspace(0,5.0,N)
for n in range(1,N+1):
    m.q[n,'x'].value = guide[n-1] + np.random.uniform(-0.25,0.25)
    m.q[n,'y'].value  = np.random.uniform(0.5,1.1)
    m.q[n,'theta_b'].value  = np.random.uniform(-np.pi/8,np.pi/8)
    m.q[n,'theta_l'].value  = np.random.uniform(-np.pi/4,np.pi/4)
    m.q[n,'r'].value  = np.random.uniform(0.0,0.5)
    
    m.GRF[n,'Y','ps'].value = np.random.uniform(0.0,1)
    m.GRF[n,'X','ps'].value = m.mu.value*m.GRF[n,'Y','ps'].value

## Tax Day Sprint
Since this model is a bit more resiliant, I'm going to give it a slightly trickier task: the tax day sprint. The previous hopper executed a "missing the boat" sprint - it just had to reach 5 metres as fast as possible and we didn't care what it was doing once it got there. Picture the robot bouncing as fast as it can to the end of the jetty and then leaping onto a boat that's already sailing away... perhaps while being pursued by ringwraiths after being ambushed at an inn. Clearly, the only thing that matters is that it ends up on the boat.

For the tax day sprint, the situation we're picturing is that our dude is late to file its taxes and has to hop rapidly to the mail box to post them. (Yes, this world where monopedal robots are advanced enough to be worthy of SARS's attention still uses conventional snail mail.) While sprinting up to the box and slam-dunking its forms in would certainly be the cooler approach, the more practical one would involve coming to a halt in an upright position. Long story short, the tax day condition means we care about the robot's final position and velocity, not just the distance it reaches. There are varying degrees of strictness, but at the very least, the foot should be on the ground, the tilt angle of the body should be within some margin (or the height of the torso above some level) an the horizontal velocity should be zero.

If you're wondering where these weird task names come from, check out <a href = "https://www.researchgate.net/profile/Monica_Daley/publication/301404270_Do_limit_cycles_matter_in_the_long_run_Stable_orbits_and_sliding-mass_dynamics_emerge_in_task-optimal_locomotion/links/57295a7208ae057b0a03434a/Do-limit-cycles-matter-in-the-long-run-Stable-orbits-and-sliding-mass-dynamics-emerge-in-task-optimal-locomotion.pdf">this paper</a> by ATRIAS team member, RAM Group collaborator and *Survivor: David vs. Goliath* star Christian Hubicki. (His publications are always worth checking out - come for the science, stay for the puns :) )


In [ ]:
# SPRINT --------------------------------------------------------------------------------------------------------------------
# tax day

# initial condition
m.q[1,'x'].value = 0.0
m.q[1,'theta_b'].value = 0.0
m.q[1,'theta_l'].value = 0.0

m.q[1,'x'].fixed = True
m.q[1,'theta_b'].fixed = True
m.q[1,'theta_l'].fixed = True

m.footp[1,'Y'].value = 0.0
m.footp[1,'Y'].fixed = True

for dof in DOFs:
    m.dq[1,dof].value = 0.0
    m.dq[1,dof].fixed = True
      
# final condition
m.q[N,'x'].setlb(5.0)

m.dq[N,'x'].value = 0.0
m.dq[N,'x'].fixed = 0.0
m.dq[N,'theta_b'].value = 0.0
m.dq[N,'theta_b'].fixed = 0.0
m.dq[N,'theta_l'].value = 0.0
m.dq[N,'theta_l'].fixed = 0.0

m.q[N,'theta_b'].value = 0.0
m.q[N,'theta_l'].value = 0.0
m.q[N,'theta_b'].fixed = True
m.q[N,'theta_l'].fixed = True

m.footp[N,'Y'].value = 0.0
m.footp[N,'Y'].fixed = True
    
#m.pprint()

In [ ]:
# solving
#opt = SolverFactory('ipopt') # standard issue, garden variety ipopt

opt = SolverFactory('ipopt',executable = 'C:/cygwin64/home/Stacey/CoinIpopt/build/bin/ipopt.exe')
opt.options["linear_solver"] = 'ma86'

# solver options
opt.options["expect_infeasible_problem"] = 'yes'
#pt.options["linear_system_scaling"] = 'none'
#opt.options["mu_strategy"] = "adaptive"
opt.options["print_level"] = 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
opt.options["max_iter"] = 30000 # maximum number of iterations
opt.options["max_cpu_time"] = 600 # maximum cpu time in seconds
opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
    
results = opt.solve(m, tee = True) 

In [ ]:
print(results.solver.status) # tells you if the solver had any errors/ warnings
print(results.solver.termination_condition) # tells you if the solution was (locally) optimal, feasible, or neither.

penalty_sum = 0
for n in range(1,N+1):
    for gc in ground_constraints:
        penalty_sum += m.ground_penalty[n,gc].value
    for jc in joint_constraints:
        for j in joints:
            penalty_sum += m.joint_penalty[n,j,jc].value

print penalty_sum

#m.h.pprint() 

In [ ]:
#animate it
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1) #create axes
ax1.set_aspect('equal')

def plot_robot(i,m,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-1,10])
    ax.set_ylim([0,3])
    
    #plot body
    body_xb = m.q[i,'x'].value - 0.5*m.len[('body',1)]*cos(m.q[i,'theta_b'].value)
    body_yb = m.q[i,'y'].value - 0.5*m.len[('body',1)]*sin(m.q[i,'theta_b'].value)
    body_xf = m.q[i,'x'].value + 0.5*m.len[('body',1)]*cos(m.q[i,'theta_b'].value)
    body_yf = m.q[i,'y'].value + 0.5*m.len[('body',1)]*sin(m.q[i,'theta_b'].value)  
    ax.plot([body_xb,body_xf],[body_yb,body_yf],color='xkcd:black')
      
    #plot leg 1
    thA = m.q[i,'theta_b'].value+m.q[i,'theta_l'].value
    leg1_xt = m.q[i,'x'].value
    leg1_yt = m.q[i,'y'].value
    leg1_xb = m.q[i,'x'].value + m.len[('leg',1)]*sin(thA)
    leg1_yb = m.q[i,'y'].value - m.len[('leg',1)]*cos(thA)
    ax.plot([leg1_xt,leg1_xb],[leg1_yt,leg1_yb],color='xkcd:black')
    
    #plot leg 2
    Lt = 0.5*m.len[('leg',1)] + m.q[i,'r'].value - 0.5*m.len[('leg',2)]
    Lb = 0.5*m.len[('leg',1)] + m.q[i,'r'].value + 0.5*m.len[('leg',2)]
    leg2_xt = m.q[i,'x'].value + Lt*sin(thA)
    leg2_yt = m.q[i,'y'].value - Lt*cos(thA)
    leg2_xb = m.q[i,'x'].value + Lb*sin(thA)
    leg2_yb = m.q[i,'y'].value - Lb*cos(thA)
    ax.plot([leg2_xt,leg2_xb],[leg2_yt,leg2_yb],color='xkcd:black')
    
update = lambda i: plot_robot(i,m,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=True)

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook
